## Imports

In [1]:
import torch
import torch.nn as nn
import numpy as np
import joblib
import os
import pandas as pd

# We need to define the model architecture again so we can load the weights into it
class Cnn1D(nn.Module):
    def __init__(self, num_classes=4):
        super(Cnn1D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=32, stride=4, padding=14),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=16, stride=2, padding=7),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=8, stride=1, padding=3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=448, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=128, out_features=num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

print("Imports and Cnn1D class definition complete.")

Imports and Cnn1D class definition complete.


## Load Artifacts
This cell loads the trained model and the scaler that we saved earlier.



In [4]:
# --- Load Artifacts ---

# Define paths to your artifacts within the streamlit_app folder
APP_FOLDER_PATH = '../streamlit_app'
MODEL_PATH = os.path.join(APP_FOLDER_PATH, 'best_1d_cnn_model.pth')
SCALER_PATH = os.path.join(APP_FOLDER_PATH, 'standard_scaler.joblib')

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = Cnn1D(num_classes=4).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval() # Set model to evaluation mode

# Load the scaler
scaler = joblib.load(SCALER_PATH)

# Define class names (ensure this order matches your label encoding)
CLASS_NAMES = ['Ball Fault', 'Inner Race Fault', 'Normal', 'Outer Race Fault']


print(f"Model loaded and set to '{device}' device.")
print(f"Scaler loaded: {scaler}")
print("Ready to make predictions.")

Model loaded and set to 'cuda' device.
Scaler loaded: StandardScaler()
Ready to make predictions.


## The Prediction Function
This is the core function. It encapsulates the entire process from reading a file to making a prediction.

In [5]:
def predict_signal(signal_path, model, scaler, device):
    """
    Loads a signal from a CSV, preprocesses it, and returns the model's prediction.
    
    Args:
        signal_path (str): The full path to the signal CSV file.
        model (torch.nn.Module): The loaded PyTorch model.
        scaler (StandardScaler): The loaded scikit-learn scaler.
        device (torch.device): The device to run inference on ('cuda' or 'cpu').
        
    Returns:
        tuple: A tuple containing (predicted_class_name, probabilities_dict)
    """
    # 1. Load the signal data
    signal = pd.read_csv(signal_path, header=None).values.flatten().astype(np.float32)
    
    # 2. Preprocess: Segment, Scale, and Convert to Tensor
    # We only analyze the first 2048 samples, as our model was trained on this size
    if len(signal) < 2048:
        return "Error: Signal is too short. Needs at least 2048 samples.", {}
        
    segment = signal[:2048]
    
    # The scaler expects a 2D array, so we reshape it
    segment_scaled = scaler.transform(segment.reshape(1, -1))
    
    # Convert to a PyTorch tensor with the correct shape (1, 1, 2048)
    segment_tensor = torch.tensor(segment_scaled, dtype=torch.float32).unsqueeze(0).to(device)
    
    # 3. Make Prediction
    with torch.no_grad():
        outputs = model(segment_tensor)
        # Apply softmax to get probabilities
        probabilities = nn.functional.softmax(outputs, dim=1)
        
        # Get the top class
        top_prob, top_class_idx = torch.max(probabilities, 1)
        predicted_class_name = CLASS_NAMES[top_class_idx.item()]

    # 4. Format probabilities for output
    probabilities_dict = {CLASS_NAMES[i]: prob.item() for i, prob in enumerate(probabilities[0])}
    
    return predicted_class_name, probabilities_dict

print("Prediction function is defined.")

Prediction function is defined.


## Test the Function
Now, let's use the function to test the four sample signals we created at the end of `02_preprocessing_feature_engineering_and_augmentation.ipynb`.

In [6]:
# --- Test the Prediction Function ---

# List the paths to your test signals
test_signal_paths = [
    os.path.join(APP_FOLDER_PATH, 'test_signal_normal.csv'),
    os.path.join(APP_FOLDER_PATH, 'test_signal_irf.csv'),
    os.path.join(APP_FOLDER_PATH, 'test_signal_bf.csv'),
    os.path.join(APP_FOLDER_PATH, 'test_signal_orf.csv')
]

for path in test_signal_paths:
    filename = os.path.basename(path)
    predicted_class, probs = predict_signal(path, model, scaler, device)
    
    print(f"--- Prediction for: {filename} ---")
    if "Error" in predicted_class:
        print(predicted_class)
    else:
        print(f"  Predicted Class: {predicted_class}")
        # Find the confidence for the predicted class
        confidence = probs[predicted_class]
        print(f"  Confidence: {confidence:.2%}")
        print("  Full Probabilities:", {k: f"{v:.2%}" for k, v in probs.items()})
    print("-" * 30)

--- Prediction for: test_signal_normal.csv ---
  Predicted Class: Normal
  Confidence: 100.00%
  Full Probabilities: {'Ball Fault': '0.00%', 'Inner Race Fault': '0.00%', 'Normal': '100.00%', 'Outer Race Fault': '0.00%'}
------------------------------
--- Prediction for: test_signal_irf.csv ---
  Predicted Class: Inner Race Fault
  Confidence: 100.00%
  Full Probabilities: {'Ball Fault': '0.00%', 'Inner Race Fault': '100.00%', 'Normal': '0.00%', 'Outer Race Fault': '0.00%'}
------------------------------
--- Prediction for: test_signal_bf.csv ---
  Predicted Class: Ball Fault
  Confidence: 100.00%
  Full Probabilities: {'Ball Fault': '100.00%', 'Inner Race Fault': '0.00%', 'Normal': '0.00%', 'Outer Race Fault': '0.00%'}
------------------------------
--- Prediction for: test_signal_orf.csv ---
  Predicted Class: Outer Race Fault
  Confidence: 100.00%
  Full Probabilities: {'Ball Fault': '0.00%', 'Inner Race Fault': '0.00%', 'Normal': '0.00%', 'Outer Race Fault': '100.00%'}
-------------